In [1]:
import os
import shutil
from collections import Counter

import cv2
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn
import torchvision
import torch.functional as F
from torchvision import datasets, models, transforms
from torchsummary import summary
from torch.optim.lr_scheduler import MultiStepLR

from nets.mobilenet_v2 import MobileNetV2

# Dataset Preprocessing

In [2]:
data_path = '/home/ubuntu/dataset/flower_photos/'
train_data_path = '/home/ubuntu/dataset/flower_photos/train'
val_data_path = '/home/ubuntu/dataset/flower_photos/val'

In [3]:
train_data_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize([224, 224]),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.516, 0.459, 0.434], [0.278, 0.261, 0.257])
    ])
val_data_transforms = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor(),
        transforms.Normalize([0.516, 0.459, 0.434], [0.278, 0.261, 0.257])
    ])

train_image_dataset = datasets.ImageFolder(train_data_path, train_data_transforms)
train_data_loader = torch.utils.data.DataLoader(train_image_dataset, batch_size=32, shuffle=True, num_workers=2, )
dataset_size = len(train_image_dataset)

val_image_dataset = datasets.ImageFolder(val_data_path, val_data_transforms)
val_data_loader = torch.utils.data.DataLoader(val_image_dataset, batch_size=32, shuffle=True, num_workers=2, )

class_counts = dict(Counter(sample_tup[1] for sample_tup in train_image_dataset.imgs))
class_weights = [1-(float(class_counts[class_id])/(dataset_size)) for class_id in range(5)]

print("Number of training samples:", len(train_image_dataset))
print("Number of validation samples:", len(val_image_dataset))

print("samples per class:", class_counts)
max_val = float(max(class_counts.values()))
class_weights = [max_val/num_images for class_id, num_images in class_counts.items()]
print("class_weights for each class:", class_weights)

Number of training samples: 2934
Number of validation samples: 736
samples per class: {0: 506, 1: 718, 2: 512, 3: 559, 4: 639}
class_weights for each class: [1.4189723320158103, 1.0, 1.40234375, 1.2844364937388193, 1.1236306729264476]


## Train The Model

In [5]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [8]:
epochs = 200
lr = 1e-3
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weights).cuda())
base_model = MobileNetV2()
base_model.load_state_dict(torch.load('models/mobilenet_v2.pth.tar'), strict=False)
model = base_model
model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(model.last_channel, 5),
    )
summary(model.cuda(), (3, 224, 224))
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
lr_scheduler = MultiStepLR(optimizer, milestones=[80, 160], gamma=0.1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
show_steps = 50

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [9]:
for epoch in range(epochs):
  running_loss = 0
  running_acc = 0
  samples = 0
  for step, (inputs, labels) in enumerate(train_data_loader):
    model.train()
    model.to(device)
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    _, preds = torch.max(outputs,1)
    loss = criterion(outputs, labels)
    
    loss.backward()
    optimizer.step()
    
    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_acc += torch.sum(preds == labels.data)
    samples += inputs.size(0)
    
    if step % show_steps == 0:
      loss = running_loss / samples
      acc = (running_acc.double() / samples).cpu().numpy()
      lr = get_lr(optimizer)
      print("Epoch:{0:03d} lr: {4:4f} Step:{1:04d} Train Loss: {2:4f} Train Accuracy: {3:4f}".format(epoch, step, loss, acc, lr))
  
  # update learning rate      
  lr_scheduler.step()
    
  # validation after every epoch
  running_loss = 0
  running_acc = 0
  with torch.no_grad():
    for (inputs, labels) in val_data_loader:
      model.eval()
      inputs = inputs.to(device)
      labels = labels.to(device)
      with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_acc += torch.sum(preds == labels.data)

    loss = running_loss / len(val_image_dataset)
    acc = (running_acc.double() / len(val_image_dataset)).cpu().numpy()
    print("Epoch:{0:03d} Test Loss: {1:4f} Test Accuracy: {2:4f}".format(epoch, loss, acc))

Epoch:000 lr: 0.001000 Step:0000 Train Loss: 1.630167 Train Accuracy: 0.156250
Epoch:000 lr: 0.001000 Step:0050 Train Loss: 1.201343 Train Accuracy: 0.705882
Epoch:000 Test Loss: 1.122704 Test Accuracy: 0.777174
Epoch:001 lr: 0.001000 Step:0000 Train Loss: 1.265254 Train Accuracy: 0.656250
Epoch:001 lr: 0.001000 Step:0050 Train Loss: 1.188767 Train Accuracy: 0.707108
Epoch:001 Test Loss: 1.133944 Test Accuracy: 0.767663
Epoch:002 lr: 0.001000 Step:0000 Train Loss: 1.162206 Train Accuracy: 0.718750
Epoch:002 lr: 0.001000 Step:0050 Train Loss: 1.131895 Train Accuracy: 0.770221
Epoch:002 Test Loss: 1.092053 Test Accuracy: 0.797554
Epoch:003 lr: 0.001000 Step:0000 Train Loss: 1.149212 Train Accuracy: 0.750000
Epoch:003 lr: 0.001000 Step:0050 Train Loss: 1.152928 Train Accuracy: 0.751225
Epoch:003 Test Loss: 1.083777 Test Accuracy: 0.811141
Epoch:004 lr: 0.001000 Step:0000 Train Loss: 1.149331 Train Accuracy: 0.750000
Epoch:004 lr: 0.001000 Step:0050 Train Loss: 1.146863 Train Accuracy: 0.7

Epoch:038 Test Loss: 1.033338 Test Accuracy: 0.873641
Epoch:039 lr: 0.001000 Step:0000 Train Loss: 1.097348 Train Accuracy: 0.812500
Epoch:039 lr: 0.001000 Step:0050 Train Loss: 1.055134 Train Accuracy: 0.848652
Epoch:039 Test Loss: 1.021390 Test Accuracy: 0.884511
Epoch:040 lr: 0.001000 Step:0000 Train Loss: 1.039779 Train Accuracy: 0.875000
Epoch:040 lr: 0.001000 Step:0050 Train Loss: 1.068231 Train Accuracy: 0.835172
Epoch:040 Test Loss: 1.043418 Test Accuracy: 0.847826
Epoch:041 lr: 0.001000 Step:0000 Train Loss: 1.050797 Train Accuracy: 0.843750
Epoch:041 lr: 0.001000 Step:0050 Train Loss: 1.066633 Train Accuracy: 0.835172
Epoch:041 Test Loss: 1.023838 Test Accuracy: 0.877717
Epoch:042 lr: 0.001000 Step:0000 Train Loss: 1.002593 Train Accuracy: 0.906250
Epoch:042 lr: 0.001000 Step:0050 Train Loss: 1.033186 Train Accuracy: 0.870711
Epoch:042 Test Loss: 1.053599 Test Accuracy: 0.845109
Epoch:043 lr: 0.001000 Step:0000 Train Loss: 1.069809 Train Accuracy: 0.843750
Epoch:043 lr: 0.001

Epoch:077 Test Loss: 1.029792 Test Accuracy: 0.876359
Epoch:078 lr: 0.001000 Step:0000 Train Loss: 1.004582 Train Accuracy: 0.875000
Epoch:078 lr: 0.001000 Step:0050 Train Loss: 1.039801 Train Accuracy: 0.865196
Epoch:078 Test Loss: 1.014550 Test Accuracy: 0.891304
Epoch:079 lr: 0.001000 Step:0000 Train Loss: 1.061561 Train Accuracy: 0.812500
Epoch:079 lr: 0.001000 Step:0050 Train Loss: 1.020069 Train Accuracy: 0.882353
Epoch:079 Test Loss: 1.049336 Test Accuracy: 0.855978
Epoch:080 lr: 0.000100 Step:0000 Train Loss: 1.006517 Train Accuracy: 0.906250
Epoch:080 lr: 0.000100 Step:0050 Train Loss: 1.021675 Train Accuracy: 0.883578
Epoch:080 Test Loss: 1.021734 Test Accuracy: 0.879076
Epoch:081 lr: 0.000100 Step:0000 Train Loss: 0.950644 Train Accuracy: 0.937500
Epoch:081 lr: 0.000100 Step:0050 Train Loss: 0.998286 Train Accuracy: 0.905025
Epoch:081 Test Loss: 1.009395 Test Accuracy: 0.894022
Epoch:082 lr: 0.000100 Step:0000 Train Loss: 0.976703 Train Accuracy: 0.937500
Epoch:082 lr: 0.000

Epoch:116 Test Loss: 0.987262 Test Accuracy: 0.917120
Epoch:117 lr: 0.000100 Step:0000 Train Loss: 0.965612 Train Accuracy: 0.937500
Epoch:117 lr: 0.000100 Step:0050 Train Loss: 0.955473 Train Accuracy: 0.949755
Epoch:117 Test Loss: 0.989710 Test Accuracy: 0.913043
Epoch:118 lr: 0.000100 Step:0000 Train Loss: 0.965044 Train Accuracy: 0.937500
Epoch:118 lr: 0.000100 Step:0050 Train Loss: 0.957528 Train Accuracy: 0.947917
Epoch:118 Test Loss: 0.990258 Test Accuracy: 0.915761
Epoch:119 lr: 0.000100 Step:0000 Train Loss: 0.932154 Train Accuracy: 0.968750
Epoch:119 lr: 0.000100 Step:0050 Train Loss: 0.964114 Train Accuracy: 0.939951
Epoch:119 Test Loss: 0.986111 Test Accuracy: 0.918478
Epoch:120 lr: 0.000100 Step:0000 Train Loss: 0.904872 Train Accuracy: 1.000000
Epoch:120 lr: 0.000100 Step:0050 Train Loss: 0.949311 Train Accuracy: 0.955882
Epoch:120 Test Loss: 0.991242 Test Accuracy: 0.914402
Epoch:121 lr: 0.000100 Step:0000 Train Loss: 0.996061 Train Accuracy: 0.906250
Epoch:121 lr: 0.000

Epoch:155 Test Loss: 0.991812 Test Accuracy: 0.913043
Epoch:156 lr: 0.000100 Step:0000 Train Loss: 0.959727 Train Accuracy: 0.937500
Epoch:156 lr: 0.000100 Step:0050 Train Loss: 0.943245 Train Accuracy: 0.961397
Epoch:156 Test Loss: 0.991206 Test Accuracy: 0.913043
Epoch:157 lr: 0.000100 Step:0000 Train Loss: 0.904839 Train Accuracy: 1.000000
Epoch:157 lr: 0.000100 Step:0050 Train Loss: 0.941686 Train Accuracy: 0.962010
Epoch:157 Test Loss: 0.995191 Test Accuracy: 0.908967
Epoch:158 lr: 0.000100 Step:0000 Train Loss: 0.934333 Train Accuracy: 0.968750
Epoch:158 lr: 0.000100 Step:0050 Train Loss: 0.944178 Train Accuracy: 0.959559
Epoch:158 Test Loss: 0.994097 Test Accuracy: 0.907609
Epoch:159 lr: 0.000100 Step:0000 Train Loss: 0.948518 Train Accuracy: 0.937500
Epoch:159 lr: 0.000100 Step:0050 Train Loss: 0.942315 Train Accuracy: 0.962010
Epoch:159 Test Loss: 0.996128 Test Accuracy: 0.907609
Epoch:160 lr: 0.000010 Step:0000 Train Loss: 0.904842 Train Accuracy: 1.000000
Epoch:160 lr: 0.000

Epoch:194 Test Loss: 0.989658 Test Accuracy: 0.913043
Epoch:195 lr: 0.000010 Step:0000 Train Loss: 0.953934 Train Accuracy: 0.968750
Epoch:195 lr: 0.000010 Step:0050 Train Loss: 0.927976 Train Accuracy: 0.977328
Epoch:195 Test Loss: 0.992970 Test Accuracy: 0.911685
Epoch:196 lr: 0.000010 Step:0000 Train Loss: 0.930258 Train Accuracy: 0.968750
Epoch:196 lr: 0.000010 Step:0050 Train Loss: 0.937631 Train Accuracy: 0.967525
Epoch:196 Test Loss: 0.992438 Test Accuracy: 0.910326
Epoch:197 lr: 0.000010 Step:0000 Train Loss: 1.029897 Train Accuracy: 0.875000
Epoch:197 lr: 0.000010 Step:0050 Train Loss: 0.936089 Train Accuracy: 0.968750
Epoch:197 Test Loss: 0.991122 Test Accuracy: 0.908967
Epoch:198 lr: 0.000010 Step:0000 Train Loss: 0.975482 Train Accuracy: 0.937500
Epoch:198 lr: 0.000010 Step:0050 Train Loss: 0.931136 Train Accuracy: 0.974265
Epoch:198 Test Loss: 0.992830 Test Accuracy: 0.908967
Epoch:199 lr: 0.000010 Step:0000 Train Loss: 0.949845 Train Accuracy: 0.968750
Epoch:199 lr: 0.000

In [10]:
# save weights
torch.save(model.state_dict(), "flower_mobilenetv2.pth")

## Inference on Actual Image

In [4]:
classes = sorted([f for f in os.listdir(train_data_path) if not f.startswith('.')])
print(classes)
imgs = [f for f in os.listdir(data_path) if not f.startswith('.')]
base_model = MobileNetV2()
base_model.load_state_dict(torch.load('models/mobilenet_v2.pth.tar'), strict=False)
model = base_model
model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(model.last_channel, 5),
    )
model.load_state_dict(torch.load('models/flower_mobilenetv2.pth'), strict=True)
summary(model.cuda(), (3, 224, 224))

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


RuntimeError: Error(s) in loading state_dict for MobileNetV2:
	size mismatch for classifier.1.weight: copying a param with shape torch.Size([5, 1280]) from checkpoint, the shape in current model is torch.Size([27, 1280]).
	size mismatch for classifier.1.bias: copying a param with shape torch.Size([5]) from checkpoint, the shape in current model is torch.Size([27]).

In [ ]:
for f in range(2):
    org_img = cv2.imread('daisy.jpeg.JPG')
    img = cv2.resize(org_img, (224,224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    img = img.astype('float32')
    img = img / 255
    inputs = (img - np.mean(img)) / np.std(img)
    inputs = np.transpose(inputs, (2,0,1))
    inputs = np.expand_dims(inputs, axis=0)
    inputs = torch.tensor(inputs)
    with torch.no_grad():
        model.eval()
        output = model(inputs.cuda())
        _, index =  torch.max(output, dim=1)
        index = index.cpu().numpy()[0]
        dst_path = os.path.join('result2', classes[index])
        if not os.path.exists(os.path.join('result2', classes[index])):
            os.makedirs(dst_path)
        print(index)
    print('class:', classes[index])